In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/fenago/MLEssentials2/main/datasets/WA_Fn-UseC_-Telco-Customer-Churn.csv")


In [ ]:
len(df)

In [ ]:
df.head()

In [ ]:
df.head().T

In [ ]:
df.dtypes

In [ ]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'] = df['TotalCharges'].fillna(0)

In [ ]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

string_columns = list(df.dtypes[df.dtypes == 'object'].index)

for col in string_columns:
    df[col] = df[col].str.lower().str.replace(' ', '_')

In [ ]:
df.dtypes

In [ ]:
df.churn = (df.churn== 'yes').astype(int)

In [ ]:
df.head().T

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [ ]:
df_train, df_val = train_test_split(df_train_full, test_size=0.33, random_state=11)

In [ ]:
y_train = df_train.churn.values
y_val = df_val.churn.values

In [ ]:
del df_train['churn']
del df_val['churn']

In [ ]:
# EDA
df_train_full.isnull().sum()

In [ ]:
df_train_full.churn.value_counts()

In [ ]:
1521/5634

In [ ]:
global_mean = df_train_full.churn.mean()
round(global_mean,3)

In [ ]:
# Analyze Cat and Numerical Data - 
categorical = ['gender', 'seniorcitizen', 'partner', 'dependents',
               'phoneservice', 'multiplelines', 'internetservice',
               'onlinesecurity', 'onlinebackup', 'deviceprotection',
               'techsupport', 'streamingtv', 'streamingmovies',
               'contract', 'paperlessbilling', 'paymentmethod']
numerical = ['tenure', 'monthlycharges', 'totalcharges']

In [ ]:
df_train_full[categorical].nunique()

In [ ]:
df_train_full[numerical].describe()

In [ ]:
df_train_full.corr()

In [ ]:
female_mean = df_train_full[df_train_full.gender == 'female'].churn.mean()
print('gender == female:', round(female_mean, 3))

male_mean = df_train_full[df_train_full.gender == 'male'].churn.mean()
print('gender == male:  ', round(male_mean, 3))

In [ ]:
partner_mean = df_train_full[df_train_full.partner == 'yes'].churn.mean()
print('partner == yes:', round(partner_mean, 3))

partner_mean = df_train_full[df_train_full.partner == 'no'].churn.mean()
print('partner == no:  ', round(partner_mean, 3))

In [ ]:
for cat in categorical:
    print(cat)
    for val in df_train_full[cat].unique():
        print("\t", val, end="")
        mean = df_train_full[df_train_full[cat] == val].churn.mean()
        print("t",val, round(mean, 3))

In [ ]:
df_group = df_train_full.groupby(by='gender').churn.agg(['mean'])
df_group['diff'] = df_group['mean'] - global_mean
df_group['risk'] = df_group['mean'] / global_mean
df_group

In [ ]:
from IPython.display import display

In [ ]:
global_mean = df_train_full.churn.mean()
global_mean

In [ ]:
for col in categorical:
    df_group = df_train_full.groupby(by=col).churn.agg(['mean'])
    df_group['diff'] = df_group['mean'] - global_mean
    df_group['risk'] = df_group['mean'] / global_mean
    display(df_group)

In [ ]:
from sklearn.metrics import mutual_info_score

In [ ]:
def calculate_mi(series):
    return mutual_info_score(series, df_train_full.churn)

df_mi = df_train_full[categorical].apply(calculate_mi)
df_mi = df_mi.sort_values(ascending=False).to_frame(name='MI')


display(df_mi.head(27))
display(df_mi.tail())

In [ ]:
df_train_full[numerical].corrwith(df_train_full.churn).to_frame('correlation')

In [ ]:
df_train_full.groupby(by='churn')[numerical].mean()

In [ ]:
from sklearn.feature_extraction import DictVectorizer

In [ ]:
train_dict = df_train[categorical + numerical].to_dict(orient='records')

In [ ]:
type(train_dict)

In [ ]:
train_dict[0]

In [ ]:
dv = DictVectorizer(sparse=False)
dv.fit(train_dict)

In [ ]:
X_train = dv.transform(train_dict)

In [ ]:
X_train.shape

In [ ]:
dv.get_feature_names()

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model = LogisticRegression(solver='liblinear', random_state=1)
model.fit(X_train, y_train)

In [ ]:
val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [ ]:
model.predict_proba(X_val)

In [ ]:
df_val.loc[20]

In [ ]:
y_pred = model.predict_proba(X_val)[:, 1]
y_pred

In [ ]:
churn = y_pred > 0.5

In [ ]:
(y_val == churn).mean()

In [ ]:
# Interpret the model

In [ ]:
model.intercept_[0]

In [ ]:
dict(zip(dv.get_feature_names(), model.coef_[0].round(3)))

In [ ]:
customer = {
    'customerid': '8879-zkjof',
    'gender': 'female',
    'seniorcitizen': 0,
    'partner': 'no',
    'dependents': 'no',
    'tenure': 41,
    'phoneservice': 'yes',
    'multiplelines': 'no',
    'internetservice': 'dsl',
    'onlinesecurity': 'yes',
    'onlinebackup': 'no',
    'deviceprotection': 'yes',
    'techsupport': 'no',
    'streamingtv': 'yes',
    'streamingmovies': 'yes',
    'contract': 'one_year',
    'paperlessbilling': 'yes',
    'paymentmethod': 'bank_transfer_(automatic)',
    'monthlycharges': 79.85,
    'totalcharges': 3320.75,
}

In [ ]:
X_test = dv.transform([customer])
model.predict_proba(X_test)[0, 1]

In [ ]:
print(list(X_test[0]))

In [ ]:
customer = {
    'gender': 'female',
    'seniorcitizen': 1,
    'partner': 'no',
    'dependents': 'no',
    'phoneservice': 'yes',
    'multiplelines': 'yes',
    'internetservice': 'fiber_optic',
    'onlinesecurity': 'no',
    'onlinebackup': 'no',
    'deviceprotection': 'no',
    'techsupport': 'no',
    'streamingtv': 'yes',
    'streamingmovies': 'no',
    'contract': 'month-to-month',
    'paperlessbilling': 'yes',
    'paymentmethod': 'electronic_check',
    'tenure': 1,
    'monthlycharges': 85.7,
    'totalcharges': 85.7
}

In [ ]:
X_test = dv.transform([customer])
model.predict_proba(X_test)[0, 1]